# Module Perceptron
___

In [1]:
# On initialise les paramètres du perceptron
def initParam(n):
    W=np.random.randn(n,1)
    b=np.random.randn(1)
    return (W,b)

## Construction générique

In [2]:
# On définit notre modèle avec la fonction sigmoïde
def perceptron(X,W,b):
    
    z = (X.dot(W)+ b)
    A = 1 / (1+np.exp(-z))
    return A

### Erreur quadratique

In [3]:
#ErreurQ
def errQ(A,y):
    m=len(y)
    return np.sum((A-y)**2)/m

In [4]:
#rapport avec la gradient
# On détermine le gradient de l'erreur quadratique
def gradient_erreurQ(A,X,y):
    
    m = len(y)
    db = np.sum( (A-y) * A * (1-A) ) /m
    dW = np.dot(X.T,(A-y) * A * (1-A) ) /m
    
    return (dW,db)

### Log loss

In [5]:
#rapport avec le log_loss
def logloss(A,y):
    eps=1e-15
    m=len(y)
    return -(np.sum(y*np.log(A+eps)+(1-y)*np.log(1-A+eps)))/m

In [6]:
def gradient_log_loss(A,X,y):
    
    m = len(y)
    db = np.sum( (A-y)) /m
    dW = np.dot(X.T,(A-y)) /m
    
    return (dW,db)

## Descente de gradient

In [7]:
# on met à jour les paramètres par descente de gradient
def mAj(dW,db,W,b,alpha):
    
    b = b - alpha * db
    w = W - alpha * dW 
    
    return (w,b)

In [8]:
# X les données en entrées
# selecteur permet de choisir entre : -log_loss (maximisation de la vraissemblance) -erreur_quadratique (minimisation de l'erreur quadratique)
# y les résultats associé aux données
def module_perceptron(W,b , X,y,selecteur,alpha=0.01, iter=100):
    if selecteur == 'log_loss':
        eps=1e-15
        m=len(y)
        for i in range(iter):
            #definition du perceptron
            A = perceptron(X,W,b)
            '''
            #calcul de la moyenne d'echec
            if i%2==0:
                perf = -(np.sum(y*np.log(A+eps)+(1-y)*np.log(1-A+eps)))/m
            '''
            #calcul du gradient
            db = np.sum( (A-y)) /m
            dW = np.dot(X.T,(A-y)) /m
            
            #mise a jour de l'erreur
            b = b - alpha * db
            W = W - alpha * dW 
        return W,b
    elif selecteur == 'erreur_quadratique':
        m=len(y)
        for i in range(iter):
            #definition du perceptron
            A = perceptron(X,W,b)
            '''
            #calcul de la moyenne d'echec
            if i%2==0:
                perf = np.sum((A-y)**2)/m
            '''
            #calcul du gradient
            db = np.sum( (A-y) * A * (1-A) ) /m
            dW = np.dot(X.T,(A-y) * A * (1-A) ) /m
            
            #mise a jour de l'erreur
            b = b - alpha * db
            w = W - alpha * dW 
        return W,b
    else:
        print("Unknow parameter")
        raise

## Fonction de prédiction

In [9]:
def predict(image,W,b):
    return (perceptron(image,W,b) > 0.5)

# Fonctions d'affichage

In [1]:
def affichage_iter(nb,W,b,selecteur,X_train,y_train,X_test,y_test):
    if nb < 100:
        print("Il faut augmenter le nombre d'iteration - 100 est le minimum")
        raise
        
    
    alpha = 0.01
    eps=1e-15
    m=len(y_train)
    mt=len(y_test)
    
    nb_point = 10
    
    #################### LOG LOSS
    if selecteur == 'log_loss':    
        acc_ll = []
        acc_llt = []
        cout_ll = []
        cout_llt = []

        print("MAXIMISATION DE LA VRAISSEMBLANCE")
        for i in range(100,nb):
            #definition du perceptron
            A = perceptron(X_train,W,b)

            #calcul de la moyenne d'echec
            if i%nb_point==0:
                c = -(np.sum(y_train*np.log(A+eps)+(1-y_train)*np.log(1-A+eps)))/m
                cout_ll.append(c)
                A_test = perceptron(X_test,W,b)
                ct =  -(np.sum(y_test*np.log(A_test+eps)+(1-y_test)*np.log(1-A_test+eps)))/mt
                cout_llt.append(ct)

                a = accuracy_score(y_train,predict(X_train,W,b))
                acc_ll.append(a)
                at = accuracy_score(y_test,predict(X_test,W,b))
                acc_llt.append(at)
                pass
            
            #calcul du gradient
            db = np.sum( (A-y_train)) /m
            dW = np.dot(X_train.T,(A-y_train)) /mt

            #mise a jour de l'erreur
            b = b - alpha * db
            W = W - alpha * dW
            pass

        #affichage des resultats
        plt.figure(figsize=(12,4))
        plt.subplot(1,2,1)
        plt.plot (cout_ll,label='Données d''entraînement')
        plt.plot (cout_llt,label='Données de test')
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(acc_ll,label='Données d''entraînement')
        plt.plot(acc_llt,label='Données de test')
        plt.legend()
        plt.show()
        print('Meilleure résultat sur les Données d entrainement :', max(acc_ll))
        print('Meilleure résultat sur les Données de test :', max(acc_llt))

    
    #################### ERREUR QUADRATIQUE 
    elif selecteur == 'erreur_quadratique':
        acc_eq = []
        acc_eqt = []
        cout_eq = []
        cout_eqt = []
        print("REDUCTION DE L'ERREUR QUADRATIQUE")

        for i in range(100,nb):
            #definition du perceptron
            A = perceptron(X_train,W,b)

            #calcul de la moyenne d'echec
            if i%nb_point==0:
                c = np.sum((A-y_train)**2)/m
                cout_eq.append(c) 
                A_test = perceptron(X_test,W,b)
                ct = np.sum((A_test-y_test)**2)/mt
                cout_eqt.append(ct)

                a = accuracy_score(y_train,predict(X_train,W,b))
                acc_eq.append(a)
                at = accuracy_score(y_test,predict(X_test,W,b))
                acc_eqt.append(at)
                pass
            
            #calcul du gradient
            db = np.sum( (A-y_train) * A * (1-A) ) /m
            dW = np.dot(X_train.T,(A-y_train) * A * (1-A) ) /m

            #mise a jour de l'erreur
            b = b - alpha * db
            w = W - alpha * dW 
            pass
        
        #affichage des resultats
        plt.figure(figsize=(12,4))
        plt.subplot(1,2,1)
        plt.plot (cout_eq,label='Données d''entraînement')
        plt.plot (cout_eqt,label='Données de test')
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(acc_eq,label='Données d''entraînement')
        plt.plot(acc_eqt,label='Données de test')
        plt.legend()
        plt.show()
        
        print('Meilleure résultat sur les Données d entrainement :', max(acc_eq))
        print('Meilleure résultat sur les Données de test :', max(acc_eqt))

    else:
        print("Unknow parameter")
        raise